# bot-gragas league bot project

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import speech_recognition as sr
from time import time
from PIL import Image
from mss import mss
from ahk import AHK
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

# 1. Initialize [AutoHotKey](https://pypi.org/project/ahk/) and [Multiple Screen Shots](https://python-mss.readthedocs.io/) libraries

In [2]:
ahk = AHK()
sct = mss()

# 2. Enter trained model name and window size parameters

In [3]:
model = "gragasnunuv2"
mon = {'top': 0, 'left': 0, 'width': 1280, 'height': 720}

# 3. Load config file and build a detection model

In [4]:
configs = config_util.get_configs_from_pipeline_file(os.path.join('models', model, 'pipeline.config'))
detection_model = model_builder.build(model_config=configs['model'], is_training=False)
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join('models', model, 'ckpt-4')).expect_partial()

category_index = label_map_util.create_category_index_from_labelmap(os.path.join('annotations', 'label_map.pbtxt'))

# 4. Define function for object detection

In [5]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 5. Algorithm for finding  on screen coordinates of an object with given index
<br>
&emsp; Filter array of detected objects for an element with given id, than calculate where the middle point is using passed coordinates.

In [6]:
def handleCoords(boxes, index):
    filtered_arr = [i for i in boxes if i.index == index]
    coords = filtered_arr[0].coordinates
    
    ymin = coords[0] * 720
    xmin = coords[1] * 1280
    ymax = coords[2] * 720
    xmax = coords[3] * 1280

    midWidth = xmin + (xmax - xmin) / 2
    midHeight = ymin + (ymax - ymin) / 2
    
    return midWidth, midHeight

# 6. Define functions containing game actions

In [7]:
def shotEnemy(boxes, index):
    print("shooting")
    midWidth, midHeight = handleCoords(boxes, index)
    ahk.mouse_move(midWidth, midHeight)
    ahk.key_press("q")

In [8]:
def jumpToAlly(boxes, index):
    print("jumping")
    midWidth, midHeight = handleCoords(boxes, index)
    ahk.mouse_move(midWidth, midHeight)
    ahk.key_press("w")

In [9]:
def healAlly():
    print("healing")
    ahk.key_press("e")

In [10]:
def ultimateEnemy(boxes, index):
    print("casting ultimate")
    midWidth, midHeight = handleCoords(boxes, index)
    ahk.mouse_move(midWidth, midHeight)
    ahk.key_press("r")

In [11]:
def igniteEnemy(boxes, index):
    print("casting spell ignite")
    midWidth, midHeight = handleCoords(boxes, index)
    ahk.mouse_move(midWidth, midHeight)
    ahk.key_press("d")

In [12]:
def exhaustEnemy(boxes, index):
    print("casting spell exhaust")
    midWidth, midHeight = handleCoords(boxes, index)
    ahk.mouse_move(midWidth, midHeight)
    ahk.key_press("f")

In [13]:
def buyItem(item):
    print(f"casting buying {item}")
    ahk.key_press("p")
    ahk.mouse_move(400, 135)
    ahk.click()
    ahk.type(item)
    ahk.key_press("Enter")
    ahk.mouse_move(890, 250)
    ahk.click()
    ahk.key_press("p")

In [14]:
def undoItem():
    print(f"undo shopping")
    ahk.key_press("p")
    ahk.mouse_move(460, 560)
    ahk.click()
    ahk.key_press("p")

# 7. Handle voice command input

In [15]:
def gameAction(boxes, text):
        words = text.lower().split()
        firstWord = words[0]
        secondWord = words[1]
        index = -1
        item = ""
        
        gragasNames = ["gragas", "grubas"]
        nunuNames = ["yeti", "nunu"]
        commands =["strzel", "skocz", "lecz", "super", "podpal", "zwolnij"]
        
        if secondWord in gragasNames:
            index = 0
        elif secondWord in nunuNames:
            index = 1
        else:
            item = secondWord
            
        if firstWord == "kup":
            buyItem(secondWord)
        elif firstWord == "cofnij":
            undoItem()
        elif firstWord in commands and index != -1:
            if firstWord == "strzel":
                shotEnemy(boxes, index)
            elif firstWord == "skocz":
                jumpToAlly(boxes, index)
            elif firstWord == "lecz":
                healAlly()
            elif firstWord == "super":
                ultimateEnemy(boxes, index)
            elif firstWord == "podpal"or firstWord == "odpal":
                igniteEnemy(boxes, index)
            elif firstWord == "zwolnij":
                exhaustEnemy(boxes, index)
        else:
            print("Incorrect command!")


# 8. Get audio data from a user

In [16]:
def recognize(boxes):
    r = sr.Recognizer()
    try:
        with sr.Microphone() as source:
            print("Listening....")
            audio = r.record(source, 2)
            print("Recognizing....")
        text = r.recognize_google(audio, language="PL-pl")
        print(text)
        gameAction(boxes, text)
    except Exception as e:
        print(e)
    ahk.sound_beep(frequency=440, duration=500)
    

# 9. Main function

In [23]:
while 1:
    begin_time = time()
    sct_img = sct.grab(mon)
    img = Image.frombytes(
        'RGB', (sct_img.size.width, sct_img.size.height), sct_img.rgb)
    img_bgr = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(img_bgr, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = img_bgr.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)
    
    
    boxes_array = detections['detection_boxes'][0:5]
    classes_array = detections['detection_classes'][0:5]
    
    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (1280, 720)))
    
    key = ahk.key_state('Control')
    
    class Object(object):
        pass
    
    if key:
        ahk.sound_beep(frequency=440, duration=500)
        objects_array = []
        for i in range(0, len(boxes_array)):
            exists = False
            a = Object()
            a.coordinates = boxes_array[i]
            a.index = classes_array[i]
            for j in objects_array:
                if j.index == a.index:
                    exists = True
            if exists == False:
                objects_array.append(a)

        recognize(objects_array)
        
    if cv2.waitKey(1) & 0xFF == ord('p'):
        cv2.destroyAllWindows()
        break

Listening....
Recognizing....
strzel gragas
shooting
Listening....
Recognizing....
lecz Yeti
healing
Listening....
Recognizing....
Skocz
list index out of range
